In [74]:
!pip install --quiet geopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
from geopy import distance

# distance matrix
C = [
        [0, 0.2, 0.4, 0.6, 0.3, 0.2, 0.5, 0.4, 0.6, 0.3],
        [0.2, 0, 0.2, 0.4, 0.2, 0.3, 0.4, 0.5, 0.5, 0.5],
        [0.4, 0.2, 0, 0.2, 0.2, 0.4, 0.3, 0.5, 0.5, 0.6],
        [0.6, 0.4, 0.2, 0, 0.3, 0.4, 0.2, 0.4, 0.3, 0.6],
        [0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.3, 0.4, 0.4],
        [0.2, 0.3, 0.4, 0.4, 0.2, 0, 0.3, 0.2, 0.5, 0.2],
        [0.5, 0.4, 0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.5],
        [0.4, 0.5, 0.5, 0.4, 0.3, 0.2, 0.2, 0, 0.3, 0.3],
        [0.6, 0.5, 0.5, 0.3, 0.4, 0.2, 0.2, 0.3, 0, 0.6],
        [0.3, 0.5, 0.6, 0.6, 0.4, 0.2, 0.5, 0.3, 0.6, 0]
    ]
m = [16,8,12,8,8,4,8,16,12,12]  # prices($) for fastpass
w = [20,10,15,10,10,5,10,20,15,15]  # waiting time(min)
d = [2,4,2,3,4,5,6,2,2,3]  # duration time(min)

shape = np.shape(C)
X = cp.Variable(shape, boolean=True) # 1 if there is a path between the rides
Y = cp.Variable(10, boolean=True)  # 1 if the fast pass is purchased
u = cp.Variable(10, nonneg=True)  # dummy variable to eliminate subtours

# Defining the objective function
# Multiplying 20 to calculate the time from distance according to walking speed
objective = 20*cp.sum(cp.multiply(C, X)) + cp.sum(w) - cp.sum(cp.multiply(Y,w)) + cp.sum(d)

# Defining the constraints
constraints = []

for i in range(10):
    constraints += [cp.sum(X[i, :]) == 1]
for j in range(10):
    constraints += [cp.sum(X[:, j]) == 1]
for i in range(1, 10):
    for j in range(1, 10):
        if i != j:
            constraints += [u[i]-u[j]+1 <= (n-1) * (1-X[i, j])]    
    
constraints += [cp.sum(cp.multiply(Y,m)) <= 50] # budget is $50 for purchasing fastpasses

constraints += [cp.diag(X) == 0] 
constraints += [u[1:] >= 2]
constraints += [u[1:] <= n]
constraints += [u[0] == 1]

# Solving the problem
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver=cp.GUROBI,verbose = False)

# Transforming the solution to a path
X_sol = np.argwhere(X.value==1)
order = X_sol[0].tolist()
for i in range(1, n):
    row = order[-1]
    order.append(X_sol[row,1])

# Showing the optimal result
print(Y.value)
print('The optimal path is:')
print( ' => '.join(map(str, order)))
distance = np.sum(np.multiply(C, X.value))
print('The optimal distance is:', np.round(distance,2), 'mile')
print("objective value: ", objective.value, "minutes")

[0. 1. 0. 1. 1. 1. 1. 0. 0. 1.]
The optimal path is:
0 => 1 => 4 => 2 => 3 => 6 => 8 => 7 => 9 => 5 => 0
The optimal distance is: 2.2 mile
objective value:  147.0 minutes
